In [3]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD
from keras import backend as K
import seaborn as sns
sns.set_style('white')
import os
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
import base64
from PIL import Image
from numpy import array
import numpy as np
import random
from matplotlib import pyplot as plt
from skimage import data
%matplotlib inline
np.random.seed(seed=7)
import csv
import pandas as pd
from scipy import ndimage, misc
from skimage import io
from sklearn import preprocessing



Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


Import in file paths for all the posters

In [4]:
x_train  = np.load('x_train.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')
x_test = np.load('x_test.npy')

In [9]:
#CNN deets
img_rows = 75
img_cols = 50
epochs = 10
batch_size= 512
num_classes = 12
input_shape = (1,img_rows,img_cols)


In [10]:
# create an empty network model
model = Sequential()

# --- input layer ---
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu', input_shape=(img_rows,img_cols, 1)))
# --- max pool ---
model.add(MaxPooling2D(pool_size=(2, 2)))

# --- next layer ---
# we could double the number of filters as max pool made the 
# feature maps much smaller 
# just not doing this to improve runtime
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
# --- max pool ---
model.add(MaxPooling2D(pool_size=(2, 2)))

# flatten for fully connected classification layer
model.add(Flatten())
# note that the 10 is the number of classes we have
# the classes are mutually exclusive so softmax is a good choice
# --- fully connected layer ---
model.add(Dense(64, activation='relu'))
# --- classification ---
model.add(Dense(num_classes, activation='softmax'))

# prints out a summary of the model architecture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 71, 46, 16)        416       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 35, 23, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 33, 21, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 10, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 5120)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                327744    
_________________________________________________________________
dense_4 (Dense)              (None, 12)                780       
Total para

In [11]:
# this does all necessary compiling. In tensorflow this is much quicker than in theano
# the setup is our basic categorical crossentropy with stochastic gradient decent
# we also specify that we want to evaluate our model in terms of accuracy
sgd = SGD(lr=0.1, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [ ]:
# this is now the actual training
# in addition to the training data we provide validation data
# this data is used to calculate the performance of the model over all the epochs
# this is useful to determine when training should stop
# in our case we just use it to monitor the evolution of the model over the training epochs
# if we use the validation data to determine when to stop the training or which model to save, we 
# should not use the test data, but a separate validation set. 
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

# once training is complete, let's see how well we have done
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 5000 samples, validate on 1000 samples
Epoch 1/10
5000/5000 [==============================] - 24s - loss: 11.2739 - acc: 0.2904 - val_loss: 11.4116 - val_acc: 0.2920
Epoch 2/10
5000/5000 [==============================] - 21s - loss: 11.1763 - acc: 0.3066 - val_loss: 11.4116 - val_acc: 0.2920
Epoch 3/10
